## 1. Import Required Libraries

In [ ]:
# Core libraries
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Image processing
from PIL import Image
import cv2

# Deep Learning
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG16, ResNet50, MobileNetV2
from tensorflow.keras.applications.vgg16 import preprocess_input as vgg_preprocess
from tensorflow.keras.applications.resnet50 import preprocess_input as resnet_preprocess
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input as mobilenet_preprocess

# Metrics
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Utilities
import warnings
warnings.filterwarnings('ignore')

# Set random seed
np.random.seed(42)
tf.random.set_seed(42)

print("TensorFlow version:", tf.__version__)
print("GPU Available:", len(tf.config.list_physical_devices('GPU')) > 0)
print("All libraries imported successfully!")

## 2. Dataset Exploration

In [ ]:
# Define paths
train_dir = 'train'
val_dir = 'val'
test_dir = 'test'

# Get class names
class_names = sorted(os.listdir(train_dir))
num_classes = len(class_names)

print(f"Number of classes: {num_classes}")
print(f"\nClass names: {class_names}")

In [ ]:
# Count images in each split
def count_images(directory):
    counts = {}
    total = 0
    for class_name in class_names:
        class_path = os.path.join(directory, class_name)
        if os.path.exists(class_path):
            count = len([f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))])
            counts[class_name] = count
            total += count
    return counts, total

train_counts, train_total = count_images(train_dir)
val_counts, val_total = count_images(val_dir)
test_counts, test_total = count_images(test_dir)

print("Dataset Statistics:")
print(f"Training images: {train_total}")
print(f"Validation images: {val_total}")
print(f"Test images: {test_total}")
print(f"Total images: {train_total + val_total + test_total}")

In [ ]:
# Visualize class distribution
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

# Training distribution
axes[0].bar(train_counts.keys(), train_counts.values(), color='steelblue')
axes[0].set_title('Training Set Distribution', fontsize=12, fontweight='bold')
axes[0].set_xlabel('Fish Species')
axes[0].set_ylabel('Number of Images')
axes[0].tick_params(axis='x', rotation=45)
axes[0].grid(True, alpha=0.3)

# Validation distribution
axes[1].bar(val_counts.keys(), val_counts.values(), color='salmon')
axes[1].set_title('Validation Set Distribution', fontsize=12, fontweight='bold')
axes[1].set_xlabel('Fish Species')
axes[1].set_ylabel('Number of Images')
axes[1].tick_params(axis='x', rotation=45)
axes[1].grid(True, alpha=0.3)

# Test distribution
axes[2].bar(test_counts.keys(), test_counts.values(), color='lightgreen')
axes[2].set_title('Test Set Distribution', fontsize=12, fontweight='bold')
axes[2].set_xlabel('Fish Species')
axes[2].set_ylabel('Number of Images')
axes[2].tick_params(axis='x', rotation=45)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

# Print detailed counts
print("\nDetailed class distribution:")
df_counts = pd.DataFrame({
    'Class': class_names,
    'Train': [train_counts.get(c, 0) for c in class_names],
    'Val': [val_counts.get(c, 0) for c in class_names],
    'Test': [test_counts.get(c, 0) for c in class_names]
})
df_counts['Total'] = df_counts['Train'] + df_counts['Val'] + df_counts['Test']
print(df_counts)

In [ ]:
# Visualize sample images from each class
def show_sample_images(directory, classes, samples_per_class=3):
    fig, axes = plt.subplots(len(classes), samples_per_class, figsize=(15, len(classes)*3))
    
    for i, class_name in enumerate(classes):
        class_path = os.path.join(directory, class_name)
        images = [f for f in os.listdir(class_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        
        for j in range(min(samples_per_class, len(images))):
            img_path = os.path.join(class_path, images[j])
            img = Image.open(img_path)
            
            if len(classes) == 1:
                ax = axes[j]
            else:
                ax = axes[i, j]
            
            ax.imshow(img)
            ax.axis('off')
            if j == 0:
                ax.set_title(f'{class_name}\n{img.size[0]}x{img.size[1]}', 
                           fontsize=10, fontweight='bold')
            else:
                ax.set_title(f'{img.size[0]}x{img.size[1]}', fontsize=9)
    
    plt.suptitle('Sample Images from Each Class', fontsize=14, fontweight='bold', y=0.995)
    plt.tight_layout()
    plt.show()

show_sample_images(train_dir, class_names, samples_per_class=4)

## 3. Data Preprocessing and Augmentation

In [ ]:
# Image parameters
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 32

print(f"Image size: {IMG_HEIGHT}x{IMG_WIDTH}")
print(f"Batch size: {BATCH_SIZE}")

In [ ]:
# Data augmentation for training
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Only rescaling for validation and test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Create generators
train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=True,
    seed=42
)

val_generator = val_test_datagen.flow_from_directory(
    val_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

test_generator = val_test_datagen.flow_from_directory(
    test_dir,
    target_size=(IMG_HEIGHT, IMG_WIDTH),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False
)

print(f"\nClasses: {train_generator.class_indices}")

In [ ]:
# Visualize augmented images
def show_augmented_images(generator, num_images=5):
    images, labels = next(generator)
    
    fig, axes = plt.subplots(1, num_images, figsize=(15, 3))
    
    for i in range(num_images):
        axes[i].imshow(images[i])
        class_idx = np.argmax(labels[i])
        class_name = list(generator.class_indices.keys())[class_idx]
        axes[i].set_title(f'{class_name}', fontsize=10, fontweight='bold')
        axes[i].axis('off')
    
    plt.suptitle('Augmented Training Images', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

show_augmented_images(train_generator, num_images=8)

## 4. Custom CNN Architecture

In [ ]:
# Build custom CNN model
def build_custom_cnn(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    model = models.Sequential([
        # First Convolutional Block
        layers.Conv2D(32, (3, 3), activation='relu', input_shape=input_shape, padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(32, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Second Convolutional Block
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(64, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Third Convolutional Block
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.Conv2D(128, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Fourth Convolutional Block
        layers.Conv2D(256, (3, 3), activation='relu', padding='same'),
        layers.BatchNormalization(),
        layers.MaxPooling2D((2, 2)),
        layers.Dropout(0.25),
        
        # Flatten and Dense Layers
        layers.Flatten(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
    )
    
    return model

# Create and display model
custom_cnn = build_custom_cnn()
custom_cnn.summary()

In [ ]:
# Define callbacks
early_stopping = callbacks.EarlyStopping(
    monitor='val_loss',
    patience=10,
    restore_best_weights=True,
    verbose=1
)

reduce_lr = callbacks.ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,
    patience=5,
    min_lr=1e-7,
    verbose=1
)

checkpoint = callbacks.ModelCheckpoint(
    'best_custom_cnn.keras',
    monitor='val_accuracy',
    save_best_only=True,
    verbose=1
)

print("Callbacks defined successfully!")

In [ ]:
# Train custom CNN
print("Training Custom CNN...")
history_custom = custom_cnn.fit(
    train_generator,
    epochs=30,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr, checkpoint],
    verbose=1
)

In [ ]:
# Plot training history for custom CNN
def plot_training_history(history, model_name):
    fig, axes = plt.subplots(1, 2, figsize=(15, 5))
    
    # Accuracy
    axes[0].plot(history.history['accuracy'], label='Train Accuracy')
    axes[0].plot(history.history['val_accuracy'], label='Val Accuracy')
    if 'top_3_accuracy' in history.history:
        axes[0].plot(history.history['top_3_accuracy'], label='Train Top-3 Acc', linestyle='--')
        axes[0].plot(history.history['val_top_3_accuracy'], label='Val Top-3 Acc', linestyle='--')
    axes[0].set_title(f'{model_name} - Accuracy', fontsize=12, fontweight='bold')
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Accuracy')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    # Loss
    axes[1].plot(history.history['loss'], label='Train Loss')
    axes[1].plot(history.history['val_loss'], label='Val Loss')
    axes[1].set_title(f'{model_name} - Loss', fontsize=12, fontweight='bold')
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('Loss')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()

plot_training_history(history_custom, 'Custom CNN')

## 5. Transfer Learning Models

### 5.1 VGG16

In [ ]:
# Build VGG16 model
def build_vgg16(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    base_model = VGG16(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model layers
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
    )
    
    return model

vgg16_model = build_vgg16()
print("VGG16 Model created!")
vgg16_model.summary()

In [ ]:
# Train VGG16
print("Training VGG16...")
history_vgg16 = vgg16_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

plot_training_history(history_vgg16, 'VGG16')

### 5.2 ResNet50

In [ ]:
# Build ResNet50 model
def build_resnet50(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model layers
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
    )
    
    return model

resnet_model = build_resnet50()
print("ResNet50 Model created!")
resnet_model.summary()

In [ ]:
# Train ResNet50
print("Training ResNet50...")
history_resnet = resnet_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

plot_training_history(history_resnet, 'ResNet50')

### 5.3 MobileNetV2

In [ ]:
# Build MobileNetV2 model
def build_mobilenet(input_shape=(IMG_HEIGHT, IMG_WIDTH, 3), num_classes=num_classes):
    base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=input_shape)
    
    # Freeze base model layers
    base_model.trainable = False
    
    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(512, activation='relu'),
        layers.BatchNormalization(),
        layers.Dropout(0.5),
        layers.Dense(256, activation='relu'),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation='softmax')
    ])
    
    model.compile(
        optimizer=keras.optimizers.Adam(learning_rate=0.001),
        loss='categorical_crossentropy',
        metrics=['accuracy', keras.metrics.TopKCategoricalAccuracy(k=3, name='top_3_accuracy')]
    )
    
    return model

mobilenet_model = build_mobilenet()
print("MobileNetV2 Model created!")
mobilenet_model.summary()

In [ ]:
# Train MobileNetV2
print("Training MobileNetV2...")
history_mobilenet = mobilenet_model.fit(
    train_generator,
    epochs=20,
    validation_data=val_generator,
    callbacks=[early_stopping, reduce_lr],
    verbose=1
)

plot_training_history(history_mobilenet, 'MobileNetV2')

## 6. Model Evaluation

In [ ]:
# Evaluate all models
def evaluate_model(model, generator, model_name):
    print(f"\n{'='*60}")
    print(f"Evaluating {model_name}...")
    print(f"{'='*60}")
    
    # Get predictions
    generator.reset()
    predictions = model.predict(generator, verbose=1)
    y_pred = np.argmax(predictions, axis=1)
    y_true = generator.classes
    
    # Calculate accuracy
    accuracy = accuracy_score(y_true, y_pred)
    
    # Confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    
    print(f"\n{model_name} Test Results:")
    print(f"Accuracy: {accuracy:.4f}")
    
    # Classification report
    print(f"\nClassification Report:")
    print(classification_report(y_true, y_pred, target_names=class_names))
    
    return {
        'model_name': model_name,
        'accuracy': accuracy,
        'predictions': predictions,
        'y_pred': y_pred,
        'y_true': y_true,
        'confusion_matrix': cm
    }

# Evaluate all models
results_custom = evaluate_model(custom_cnn, test_generator, 'Custom CNN')
results_vgg16 = evaluate_model(vgg16_model, test_generator, 'VGG16')
results_resnet = evaluate_model(resnet_model, test_generator, 'ResNet50')
results_mobilenet = evaluate_model(mobilenet_model, test_generator, 'MobileNetV2')

## 7. Model Comparison

In [ ]:
# Compile results
all_results = [results_custom, results_vgg16, results_resnet, results_mobilenet]

# Create comparison dataframe
comparison_df = pd.DataFrame([
    {
        'Model': result['model_name'],
        'Test Accuracy': result['accuracy']
    }
    for result in all_results
])

print("\n" + "="*60)
print("MODEL COMPARISON SUMMARY")
print("="*60)
print(comparison_df.to_string(index=False))
print("\n" + "="*60)

# Find best model
best_model_idx = comparison_df['Test Accuracy'].idxmax()
best_model_name = comparison_df.loc[best_model_idx, 'Model']
print(f"\nBest Model: {best_model_name}")
print(f"Test Accuracy: {comparison_df.loc[best_model_idx, 'Test Accuracy']:.4f}")

In [ ]:
# Visualize comparison
plt.figure(figsize=(10, 6))
plt.bar(comparison_df['Model'], comparison_df['Test Accuracy'], color='steelblue')
plt.title('Model Accuracy Comparison', fontsize=14, fontweight='bold')
plt.ylabel('Test Accuracy')
plt.ylim([0, 1])
plt.grid(True, alpha=0.3, axis='y')

# Add value labels
for i, v in enumerate(comparison_df['Test Accuracy']):
    plt.text(i, v + 0.02, f'{v:.4f}', ha='center', fontweight='bold')

plt.tight_layout()
plt.show()

## 8. Confusion Matrices

In [ ]:
# Plot confusion matrices
fig, axes = plt.subplots(2, 2, figsize=(16, 14))
axes = axes.flatten()

for idx, result in enumerate(all_results):
    cm = result['confusion_matrix']
    
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
                xticklabels=class_names, yticklabels=class_names,
                ax=axes[idx], cbar=True)
    axes[idx].set_title(f"{result['model_name']} - Confusion Matrix\n(Accuracy={result['accuracy']:.4f})", 
                       fontsize=12, fontweight='bold')
    axes[idx].set_ylabel('True Label')
    axes[idx].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

## 9. Prediction Visualization

In [ ]:
# Visualize predictions
def visualize_predictions(model, generator, class_names, num_images=12):
    generator.reset()
    images, labels = next(generator)
    predictions = model.predict(images[:num_images])
    
    fig, axes = plt.subplots(3, 4, figsize=(16, 12))
    axes = axes.flatten()
    
    for i in range(num_images):
        axes[i].imshow(images[i])
        
        true_label = class_names[np.argmax(labels[i])]
        pred_label = class_names[np.argmax(predictions[i])]
        confidence = np.max(predictions[i])
        
        color = 'green' if true_label == pred_label else 'red'
        axes[i].set_title(f'True: {true_label}\nPred: {pred_label}\nConf: {confidence:.2f}',
                         color=color, fontweight='bold', fontsize=9)
        axes[i].axis('off')
    
    plt.tight_layout()
    plt.show()

# Get best model
best_model_obj = [custom_cnn, vgg16_model, resnet_model, mobilenet_model][best_model_idx]

print(f"Predictions from Best Model ({best_model_name}):")
visualize_predictions(best_model_obj, test_generator, class_names, num_images=12)

## 10. Visualization of CNN Learned Features

In [ ]:
# Visualize feature maps from custom CNN
def visualize_feature_maps(model, image, layer_name, num_filters=16):
    # Create a model that outputs the feature maps
    layer_output = model.get_layer(layer_name).output
    feature_model = models.Model(inputs=model.input, outputs=layer_output)
    
    # Get the feature maps
    feature_maps = feature_model.predict(np.expand_dims(image, axis=0))
    
    # Plot the feature maps
    cols = 8
    rows = int(np.ceil(num_filters / cols))
    
    fig, axes = plt.subplots(rows, cols, figsize=(16, rows*2))
    axes = axes.flatten()
    
    for i in range(min(num_filters, feature_maps.shape[-1])):
        axes[i].imshow(feature_maps[0, :, :, i], cmap='viridis')
        axes[i].axis('off')
        axes[i].set_title(f'Filter {i+1}', fontsize=8)
    
    # Hide unused subplots
    for i in range(num_filters, len(axes)):
        axes[i].axis('off')
    
    plt.suptitle(f'Feature Maps from Layer: {layer_name}', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()

# Get a sample image
test_generator.reset()
sample_images, sample_labels = next(test_generator)
sample_image = sample_images[0]

# Show original image
plt.figure(figsize=(4, 4))
plt.imshow(sample_image)
plt.title('Original Image', fontweight='bold')
plt.axis('off')
plt.show()

# Visualize feature maps from different layers
try:
    visualize_feature_maps(custom_cnn, sample_image, 'conv2d', num_filters=32)
    visualize_feature_maps(custom_cnn, sample_image, 'conv2d_2', num_filters=32)
    visualize_feature_maps(custom_cnn, sample_image, 'conv2d_4', num_filters=32)
except Exception as e:
    print(f"Note: Feature map visualization requires specific layer names. Error: {e}")

## 11. Summary and Conclusions

In [ ]:
print("\n" + "="*80)
print("FISH IMAGE CLASSIFICATION PROJECT SUMMARY")
print("="*80)

print(f"\n1. Dataset Statistics:")
print(f"   - Number of classes: {num_classes}")
print(f"   - Training images: {train_total}")
print(f"   - Validation images: {val_total}")
print(f"   - Test images: {test_total}")
print(f"   - Image size: {IMG_HEIGHT}x{IMG_WIDTH}")

print(f"\n2. Models Trained:")
for i, result in enumerate(all_results, 1):
    print(f"   {i}. {result['model_name']}")

print(f"\n3. Best Model: {best_model_name}")
print(f"   - Test Accuracy: {comparison_df.loc[best_model_idx, 'Test Accuracy']:.4f}")

print(f"\n4. Key Techniques Used:")
print(f"   - Data augmentation (rotation, shift, zoom, flip)")
print(f"   - Custom CNN architecture with batch normalization")
print(f"   - Transfer learning (VGG16, ResNet50, MobileNetV2)")
print(f"   - Callbacks (EarlyStopping, ReduceLROnPlateau, ModelCheckpoint)")
print(f"   - Comprehensive evaluation and visualization")

print(f"\n5. Performance Ranking:")
ranked_df = comparison_df.sort_values('Test Accuracy', ascending=False).reset_index(drop=True)
for i, row in ranked_df.iterrows():
    print(f"   {i+1}. {row['Model']:20s} - Accuracy: {row['Test Accuracy']:.4f}")

print("\n" + "="*80)
print("PROJECT COMPLETED SUCCESSFULLY!")
print("="*80)